In [5]:
from model import *
import pyomo.environ as pyo
from unit_models.valve import SVValve
from unit_models.pid_controller import SVPIDController
from unit_models.heater import SVHeater
from idaes.core import FlowsheetBlock, MaterialBalanceType
from idaes.models.properties import iapws95
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import propagate_state
from idaes.core.solvers import get_solver
from idaes.models.control.controller import (
    ControllerType,
    ControllerMVBoundType
)
from idaes.core.util import DiagnosticsToolbox
import idaes.logger as idaeslog

def _add_inlet_pressure_step(m, time=1, value=6.0e5):
    """Add an inlet pressure step change"""
    for t in m.fs.time:
        if t >= time:
            m.fs.valve_1.inlet.pressure[t].fix(value)




def create_model(time_set=None, time_units=pyo.units.s, nfe=5, tee=False):
    """Build and initialize the flowsheet model

           valve_1   +----+
  steam ----|><|-->--| ta |    valve_2
                     | nk |-----|><|--->--- steam
                     +----+
    """
    m = pyo.ConcreteModel(name="Dynamic Steam Tank with PID Control")
    # Add flowsheet
    m.fs = FlowsheetBlock(dynamic=True, time_set=time_set, time_units=time_units)
    # Add water property parameter block
    m.fs.prop_water = iapws95.Iapws95ParameterBlock(
        phase_presentation=iapws95.PhaseType.LG
    )
    # Add valve 1
    m.fs.valve_1 = SVValve(
        dynamic=False,
        has_holdup=False,
        material_balance_type=MaterialBalanceType.componentTotal,
        property_package=m.fs.prop_water,
    )
    # Add heater model to represent a tank (close to bare control volume model).
    m.fs.tank = SVHeater(
        has_holdup=True,
        material_balance_type=MaterialBalanceType.componentTotal,
        property_package=m.fs.prop_water,
    )
    # Add valve 2
    m.fs.valve_2 = SVValve(
        dynamic=False,
        has_holdup=False,
        material_balance_type=MaterialBalanceType.componentTotal,
        property_package=m.fs.prop_water,
    )
    # Add a controller
    m.fs.ctrl = SVPIDController(
        process_var=m.fs.tank.control_volume.properties_out[:].pressure,
        manipulated_var=m.fs.valve_1.valve_opening,
        calculate_initial_integral=True,
        mv_bound_type=ControllerMVBoundType.SMOOTH_BOUND,
        controller_type=ControllerType.PI,
    )

    m.fs.ctrl.mv_lb = 0.0 # valve opening lower bound is 0. Note this isn't a variable, its' just a constant. so it doesn't show up in state vars.
    m.fs.ctrl.mv_ub = 1.0 # value opening upper bound is 1

    # The control volume block doesn't assume equilibrium, so I'll make that
    # assumption here. I don't actually expect liquid to form but who knows?
    # The phase_fraction in the control volume is volumetric phase fraction.
    @m.fs.tank.Constraint(m.fs.time)
    def vol_frac_vap(b, t):
        return (
            b.control_volume.properties_out[t].phase_frac["Vap"]
            * b.control_volume.properties_out[t].dens_mol
            / b.control_volume.properties_out[t].dens_mol_phase["Vap"]
        ) == (b.control_volume.phase_fraction[t, "Vap"])

    # Connect the models
    m.fs.v1_to_tank = Arc(source=m.fs.valve_1.outlet, destination=m.fs.tank.inlet)
    m.fs.tank_to_v2 = Arc(source=m.fs.tank.outlet, destination=m.fs.valve_2.inlet)

    
    # Add the stream constraints
    pyo.TransformationFactory("network.expand_arcs").apply_to(m.fs)


    # Do DAE discretization
    pyo.TransformationFactory("dae.finite_difference").apply_to(
        m.fs,
        nfe=nfe,
        wrt=m.fs.time,
        scheme="BACKWARD"
    )

    register_inlet_ports(m.fs)


    # Fix the derivative variables to zero at time 0 (steady state assumption)
    
    #m.fs.fix_initial_conditions() is equivalent to the following lines:
    m.fs.tank.control_volume.material_accumulation[0,"Liq","H2O"].fix(0)
    m.fs.tank.control_volume.material_accumulation[0,"Vap","H2O"].fix(0)
    m.fs.tank.control_volume.energy_accumulation[0,"Liq"].fix(0)
    m.fs.tank.control_volume.energy_accumulation[0,"Vap"].fix(0)



    # Replacements
    m.fs.valve_1.inlet.flow_mol.fix(100) # initial guess for inlet flow rate
    replace_state_var(m.fs.valve_1.inlet.flow_mol, m.fs.valve_2.outlet.pressure)
    m.fs.valve_2.outlet.pressure.fix(2.5e5)
    
    # the setpoint automatically replaces the manipulated variable (valve 1 opening fraction) when you create a PID controller.
    m.fs.ctrl.setpoint.fix(3e5) # setpoint is tank pressure of 300 kPa

    # Fix the input variables
    m.fs.valve_1.Cv.fix(0.9) # valve 1 flow coefficient
    m.fs.valve_1.inlet.enth_mol.fix(50000) # inlet fluid molar enthalpy
    m.fs.valve_1.inlet.pressure.fix(5e5) 
    m.fs.tank.heat_duty.unfix()
    m.fs.tank.heat_duty.fix(0) # no heat transfer to tank
    m.fs.tank.control_volume.volume.fix(2.0) # 2 m^3 tank volume

    m.fs.valve_2.Cv.fix(2) # valve 2 flow coefficient
    m.fs.valve_2.valve_opening.fix(1) # fix valve 2 opening to full open
    
    m.fs.ctrl.gain_p.fix(1e-8) # proportional gain
    m.fs.ctrl.gain_i.fix(1e-6) # integral error gain
    m.fs.ctrl.mv_ref.fix(0) # controller bias

    pprint_replacements(m.fs)

    m.fs.valve_1.valve_opening[0.0].fix(0.9) # We still need to set the initial valve opening. this is an initial condition. Doing it here as pprint replacements doesn't really work for initial conditions.

    print("degrees of freedom", degrees_of_freedom(m))

    # use model diagnostics to find overconstrained set (debugging purposes, uncomment if required)
    # dt = DiagnosticsToolbox(m)
    # dt.display_overconstrained_set()
    # dt.display_underconstrained_set()

    # Initialize the model
    solver = get_solver(options={"max_iter": 50})
    m.fs.valve_1.initialize(outlvl=idaeslog.INFO)
    propagate_state(m.fs.v1_to_tank)
    m.fs.tank.initialize(outlvl=idaeslog.INFO)
    propagate_state(m.fs.tank_to_v2)
    m.fs.valve_2.initialize(outlvl=idaeslog.INFO)


    assert degrees_of_freedom(m) == 0

    # Return the model and solver
    return m, solver


# Create a model for the 0 to 12 sec time period
m, solver = create_model(time_set=[0, 9], nfe=30, tee=False)
print("Solving the model...")
solver.solve(m, tee=True)




Replacements in block fs:
(Variable -> Replaced State Var)
  fs.ctrl.setpoint -> fs.valve_1.valve_opening
  fs.valve_2._pressure_outlet_ref -> fs.valve_1._flow_mol_inlet_ref

Unreplaced state variables in block fs:
  fs.valve_1.Cv
  fs.valve_1._enth_mol_inlet_ref
  fs.valve_1._pressure_inlet_ref
  fs.tank.heat_duty
  fs.tank.control_volume.volume
  fs.tank.control_volume.energy_accumulation[0.0,Liq]
  fs.tank.control_volume.energy_accumulation[0.0,Vap]
  fs.valve_2.valve_opening
  fs.valve_2.Cv
  fs.ctrl.gain_p
  fs.ctrl.gain_i
  fs.ctrl.mv_ref
degrees of freedom 0
2025-11-18 14:36:51 [INFO] idaes.init.fs.valve_1: Initialization Complete: optimal - Optimal Solution Found
2025-11-18 14:36:51 [INFO] idaes.init.fs.tank.control_volume: Initialization Complete
2025-11-18 14:36:51 [INFO] idaes.init.fs.tank: Initialization Complete: optimal - Optimal Solution Found
2025-11-18 14:36:52 [INFO] idaes.init.fs.valve_2: Initialization Complete: optimal - Optimal Solution Found
Solving the model...


{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 1048, 'Number of variables': 1048, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 2.401132822036743}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [8]:
m.fs.valve_1.report()
m.fs.tank.report()
m.fs.valve_2.report()


Unit : fs.valve_1                                                          Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key               : Value       : Units                                 : Fixed : Bounds
      Mechanical Work :      0.0000 :                                  watt : False : (None, None)
              Opening :     0.90000 :                         dimensionless :  True : (0, 1)
      Pressure Change : -2.1477e+05 :                                pascal : False : (None, None)
       Pressure Ratio :     0.57046 :                         dimensionless : False : (None, None)
    Valve Coefficient :     0.90000 : meter ** 0.5 * mole / kilogram ** 0.5 :  True : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                         Units           Inlet     Outlet  
    Molar Flow          mole / second     375